In [2]:
from __future__ import annotations
from dataclasses import dataclass

import jax
import jax.numpy as jnp
from jax.tree_util import register_dataclass
import jaxtyping
import optax
import matplotlib.pyplot as plt
import numpy as np

import hodel
import hodel.dismech as dismech

jax.config.update("jax_enable_x64", True)


def from_geo(geo):
    conn = dismech.Connectivity.from_geo(geo)
    state = dismech.StaticState.from_geo(geo, conn)

    node_dofs = dismech.map_node_to_dof(
        jnp.asarray(geo.bend_twist_springs[:, [0, 2, 4]], dtype=jnp.int32)
    )

    l0 = jnp.linalg.norm(state.q[node_dofs[:, 1]] - state.q[node_dofs[:, 0]], axis=1)
    l1 = jnp.linalg.norm(state.q[node_dofs[:, 2]] - state.q[node_dofs[:, 1]], axis=1)
    l_k = jnp.stack([l0, l1], axis=1)

    return (
        conn,
        state,
        jax.vmap(dismech.ParametrizedDERTriplet.init, (0, 0, 0, 0, 0, 0, None))(
            node_dofs,
            conn.triplet_edge_dofs,
            conn.triplet_dir_dofs,
            conn.triplet_signs,
            l_k,
            jnp.arange(node_dofs.shape[0])[..., None],
            state,
        ),
    )


EA = jnp.array([6.28318531, 6.28318531])
EI = jnp.array([1.57079633e-06, 1.57079633e-06])
GJ = jnp.array([1.04719755e-06])
Theta_star = jnp.concat([EA, EI, GJ])
geo = dismech.Mesh.from_txt("rod.txt")
conn, state, triplets = from_geo(geo)
idx_b = jnp.array(
    [
        0,
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        geo.nodes.shape[0] * 3 - 9,
        geo.nodes.shape[0] * 3 - 8,
        geo.nodes.shape[0] * 3 - 7,
        geo.nodes.shape[0] * 3 - 6,
        geo.nodes.shape[0] * 3 - 5,
        geo.nodes.shape[0] * 3 - 4,
        geo.nodes.shape[0] * 3 - 3,
        geo.nodes.shape[0] * 3 - 2,
        geo.nodes.shape[0] * 3 - 1,
        geo.nodes.shape[0] * 3,
        geo.nodes.shape[0] * 3 + 1,
        state.q.shape[0] - 2,
        state.q.shape[0] - 1,
    ]
)
idx_f = jnp.setdiff1d(jnp.arange(state.q.shape[0]), idx_b)
mass = jnp.concat(
    [
        jnp.ones(geo.nodes.shape[0] * 3) * 1.88495559e-05,
        jnp.ones(geo.edges.shape[0]) * 9.42477796e-12,
    ]
)


@register_dataclass
@dataclass(frozen=True)
class TripletAux:
    """parametrized external force."""

    top: dismech.Connectivity
    idx_f: jax.Array
    idx_b: jax.Array


def get_gravity(mass: jax.Array, g: float = -9.81):
    return mass * jnp.concat(
        [
            jnp.tile(jnp.array([0.0, 0.0, g]), geo.nodes.shape[0]),
            jnp.zeros(geo.edges.shape[0]),
        ]
    )


def get_W(lambda_: jax.Array, aux: TripletAux):
    return get_gravity(mass)[aux.idx_f]


def fixed_0(lambda_: jax.Array, aux: TripletAux) -> jax.Array:
    return state.q[aux.idx_b] + lambda_ * jnp.concat(
        [
            jnp.array([0.05, 0.0, 0, 0.05, 0.0, 0, 0.05, 0.0, 0]),
            jnp.zeros(aux.idx_b.shape[0] - 13),
            jnp.array([1.0, 1.0, 0.0, 0.0]),
        ]
    )


def update_state(
    xf: jax.Array, xb: jax.Array, aux: TripletAux, carry: dismech.StaticState
) -> dismech.StaticState:
    q = jnp.empty((aux.idx_f.shape[0] + aux.idx_b.shape[0]), xf.dtype)
    q = q.at[aux.idx_f].set(xf).at[aux.idx_b].set(xb)
    carry_new = carry.update(q, aux.top)
    return carry_new


def get_q(xf, xb, aux):
    q = jnp.empty((aux.idx_f.shape[0] + aux.idx_b.shape[0]), xf.dtype)
    return q.at[aux.idx_f].set(xf).at[aux.idx_b].set(xb)


def get_batch_energy(xf, xb, Theta, aux, carry):
    q = get_q(xf, xb, aux)
    state = carry.update(q, aux.top)
    return jnp.sum(jax.vmap(lambda t: t.get_energy(state, Theta))(triplets))


xf0 = state.q[idx_f]
aux = TripletAux(conn, idx_f, idx_b)
sim = hodel.HODEL(
    get_batch_energy, get_W_fn=get_W, get_xb_fn=fixed_0, carry_fn=update_state
)

In [3]:
lambdas = jnp.linspace(0, 1.0, 100)
xf_star = sim.sim(lambdas, xf0, Theta_star, aux, state, nsteps=10)
xb_star = jax.vmap(fixed_0, (0, None))(lambdas, aux)
qs = jax.vmap(get_q, (0, 0, None))(xf_star, xb_star, aux)
dismech.animate(lambdas, qs, conn)

In [ ]:
lambdas = jnp.linspace(0, 1.0, 10)
xf_star = sim.sim(lambdas, xf0, Theta_star, aux, state)
key = jax.random.PRNGKey(0)
Theta0 = jax.random.uniform(key, [5], minval=1e-2, maxval=1e1)
lr = 1e1
nepochs = 500

final_Theta, L = sim.learn(
    lambdas,
    xf0,
    xf_star,
    Theta0,
    aux,
    state,
    optim=optax.adam(lr),
    nepochs=nepochs,
)

print("True Theta:", Theta_star)
print("Theta0:", Theta0)
print("Final Theta:", final_Theta)

plt.plot(L)
plt.yscale("log")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.show()

[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
[ 0. nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  0.]
